In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

n_users = 6040
n_movies = 3952

def get_train_data():
    return pd.read_csv('titanic/train.csv', engine='python')

def get_test_data():
    return pd.read_csv('titanic/test.csv', engine='python')

def split_train_val_test(ratings, train=0.8, val=0.1):
    shuffled = np.random.RandomState(0).permutation(ratings.index)
    n_train = int(len(shuffled) * train)
    n_val = int(len(shuffled) * val)
    i_train, i_val, i_test = shuffled[:n_train], shuffled[n_train: n_train + n_val], shuffled[-n_val:]
    return ratings.loc[i_train], ratings.loc[i_val], ratings.loc[i_test]

def get_dense_array(ratings_df):
    ratings = np.zeros((n_users, n_movies))
    ratings[ratings_df['user_id'] - 1, ratings_df['movie_id'] - 1] = ratings_df['rating']
    return ratings

In [7]:
train = get_train_data()
train.drop(columns = ['PassengerId', 'Name', 'Ticket'])
# movies = get_movie_data()
# ratings = get_rating_data()
# train_ratings_df, val_ratings_df, test_ratings_df = split_train_val_test(ratings)
# train_ratings, val_ratings, test_ratings = get_dense_array(train_ratings_df), get_dense_array(val_ratings_df), get_dense_array(test_ratings_df)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,7.2500,NaN,S
1,1,1,female,38.0,1,0,71.2833,C85,C
2,1,3,female,26.0,0,0,7.9250,NaN,S
3,1,1,female,35.0,1,0,53.1000,C123,S
4,0,3,male,35.0,0,0,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,NaN,S
887,1,1,female,19.0,0,0,30.0000,B42,S
888,0,3,female,NaN,1,2,23.4500,NaN,S
889,1,1,male,26.0,0,0,30.0000,C148,C


In [8]:
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


# 3.2 Neural Network

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

In [13]:
from sklearn.preprocessing import MinMaxScaler
def min_max_normalize(df, ignore_cols=['survived, PassengerId']):
    ''' Applies min-max normalization to numerical features in df.

    Values are scaled to [-1, 1]
    Optionally ignores columns in ignore_cols
    '''
    numeric_cols = df.select_dtypes('number').columns
    if ignore_cols is not None:
        assert set(ignore_cols) <= set(numeric_cols)
        numeric_cols = set(numeric_cols) - set(ignore_cols)
    numeric_df = df[numeric_cols]
    scaler = MinMaxScaler(feature_range=(-1, 1))
    return pd.DataFrame(scaler.fit_transform(numeric_df), columns=numeric_cols)

In [16]:
# Desired features:
# 1. one hot encode gender, 
# 2. Calculate average rating for a movie and user
def process_features(df_old):
    df = df_old.copy()
    df_num = min_max_normalize(df)
    #PassengerId - Survived - Pclass - Name - Sex - Ae- SibSp - Parch - Ticket - Fare - Cabin - Embarked
    df['age'] = df_num.Age
    df['Fare']= df_num.Fare
    df['SibSp'] = df_num.SibSp
    df['Parch'] = df_num.Parch
    sex_categorical = pd.get_dummies(df.sex)
    pclass_categorical = pd.get_dummies(df.Pclass)
    cabin_categorical = pd.get_dummies(df.Cabin)
    embarked_categorical = pd.get_dummies(df.Embarked)
    print(sex_categorical) 
    return df

In [17]:
process_features(train)

AssertionError: 

In [ ]:
# Hyperparameters
args = {
    'width': 100,
    'depth': 4,
    'hidden_size': 50,
    'lr': 0.01,
    'momentum': 0.9,
    'epochs': 10
}

layers = []
layers.append(nn.Linear(5, args['hidden_size']))
layers.append(nn.ReLU())
for i in range(args['depth']-1):
    layers.append(nn.Linear(args['hidden_size'], args['hidden_size']))
    layers.append(nn.ReLU())
layers.append(nn.Linear(args['hidden_size'], 1))

model = nn.Sequential(layers)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.input_layer = nn.Linear(5, args['hidden_size'])
        self.hidden_layers = nn.Sequential(*[nn.Linear(args['hidden_size'], args['hidden_size']), nn.ReLU()] * (args['depth']-1))
        self.output_layer = nn.Linear(args['hidden_size'], 1)
    def forward(self, x):
        x = self.input_layer(x)
        x = nn.ReLU(x)
        x = self.hidden_layers(x)
        x = self.output_layer(x)
        return x

In [ ]:
processed_train_df = process_features(train_ratings_df)
processed_val_df = process_features(val_ratings_df)
processed_test_df = process_features(test_ratings_df)

In [ ]:
from torch.utils.data import Dataset, DataLoader
class MovieLensDataset(Dataset):
    def __init__(self, df):
        self.data = df[['age', 'avg_user_rating', 'avg_movie_rating', 'F', 'M']]
        self.labels = df[['rating']]
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return torch.tensor(self.data.iloc[idx]), torch.tensor(self.labels.iloc[idx])


train_data = MovieLensDataset(processed_train_df)
val_data = MovieLensDataset(processed_val_df)
test_data = MovieLensDataset(processed_test_df)

trainloader = DataLoader(train_data, batch_size = 256, num_workers = 0)
valoader = DataLoader(val_data, batch_size = 256, num_workers = 0)
testloader = DataLoader(test_data, batch_size = 256, num_workers = 0)

In [ ]:
import torch.optim as optim
import time
def train(trainloader, layers):
    model = nn.Sequential(*layers)
    criterion = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr = args['lr'], momentum = args['momentum'])
    for epoch in range(args['epochs']):
        epoch_loss = 0
        start = time.perf_counter()
        for i, batch in enumerate(trainloader):
            inputs, labels = batch
            optimizer.zero_grad()
            outputs = model(inputs.float())
            loss = criterion(outputs, labels.float())
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        print('Epoch {} - loss: {}'.format(epoch+1, epoch_loss))
        print('Time: {}'.format(time.perf_counter() - start))
    return model



In [ ]:
model = train(trainloader, layers)

Epoch 1 - loss: 4141.011433124542
Time: 348.93263468900113
Epoch 2 - loss: 4141.175492674112
Time: 346.5130607689971
Epoch 3 - loss: 4140.659157186747
Time: 346.35102339999867
Epoch 4 - loss: 4140.003085285425
Time: 347.3758087069982
Epoch 5 - loss: 4139.870202124119
Time: 348.1088515859992
Epoch 6 - loss: 4139.900214076042
Time: 351.2712993580026
Epoch 7 - loss: 4140.127483487129
Time: 351.24977211200166
Epoch 8 - loss: 4139.785731256008
Time: 349.48898366199865
Epoch 9 - loss: 4139.558216154575
Time: 347.6035507310007
Epoch 10 - loss: 4139.661872237921
Time: 347.62262104300316


In [ ]:
def evaluate(model, valloader):
    criterion = nn.MSELoss()
    loss = 0
    with torch.no_grad():
        for i, batch in enumerate(valloader):
            inputs, labels = batch
            outputs = model(inputs.float())
            loss += criterion(outputs, labels.float())
    return loss/len(valloader)

In [ ]:
mse = evaluate(model, valoader)
print(mse)

tensor(1.0600)


In [ ]:
test_mse = evaluate(model, testloader)
print(mse)

tensor(1.0600)


In [ ]:
class MovieLensDatasetAblation(Dataset):
    def __init__(self, df):
        self.data = df[['avg_user_rating', 'avg_movie_rating']]
        self.labels = df[['rating']]
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return torch.tensor(self.data.iloc[idx]), torch.tensor(self.labels.iloc[idx])


train_data_ablation = MovieLensDataset(processed_train_df)
val_data_ablation = MovieLensDataset(processed_val_df)
test_data_ablation = MovieLensDataset(processed_test_df)

trainloader_ablation = DataLoader(train_data_ablation, batch_size = 256, num_workers = 0)
valoader_ablation = DataLoader(val_data_ablation, batch_size = 256, num_workers = 0)
testloader_ablation = DataLoader(test_data_ablation, batch_size = 256, num_workers = 0)

In [ ]:
model_ablation = train(trainloader, layers)

Epoch 1 - loss: 4451.598682403564
Time: 341.829094217
Epoch 2 - loss: 4153.0240265131
Time: 341.310661997
Epoch 3 - loss: 4148.3260371387005
Time: 345.2031759650001
Epoch 4 - loss: 4146.374026298523
Time: 349.5462531249998
Epoch 5 - loss: 4146.16229313612
Time: 355.3534050019998
Epoch 6 - loss: 4146.315999120474
Time: 358.8211854299998
Epoch 7 - loss: 4144.445684820414
Time: 353.86359875000016
Epoch 8 - loss: 4143.0946026444435
Time: 347.375252068
Epoch 9 - loss: 4142.292069166899
Time: 337.717256937
Epoch 10 - loss: 4142.6262129843235
Time: 337.6882162750003


In [ ]:
val_mse_ablation = evaluate(model_ablation, valoader)
print(val_mse_ablation)
test_mse_ablation = evaluate(model_ablation, testloader)
print(test_mse_ablation)

tensor(1.0594)
tensor(1.0594)


In [ ]:
args['epochs'] = 2
model = train(trainloader, layers)
print(2)
print('Val:')
print(evaluate(model, valoader))
print('Test:')
print(evaluate(model, testloader))

Epoch 1 - loss: 4140.293397843838
Time: 337.1380207760003
Epoch 2 - loss: 4140.029753744602
Time: 340.45485344499957
2
Val:
tensor(1.0602)
Test:
tensor(1.0602)


In [ ]:
args['epochs'] = 4
model = train(trainloader, layers)
print(4)
print('Val:')
print(evaluate(model, valoader))
print('Test:')
print(evaluate(model, testloader))

Epoch 1 - loss: 4139.735735446215
Time: 340.71909113499987
Epoch 2 - loss: 4139.335850358009
Time: 338.3462146310003
Epoch 3 - loss: 4139.791591823101
Time: 342.57236025099974
Epoch 4 - loss: 4139.2761444449425
Time: 336.70349016399996
4
Val:
tensor(1.0612)
Test:
tensor(1.0612)


In [ ]:
args['epochs'] = 6
model = train(trainloader, layers)
print(6)
print('Val:')
print(evaluate(model, valoader))
print('Test:')
print(evaluate(model, testloader))

Epoch 1 - loss: 4138.676314264536
Time: 337.9464519009998
Epoch 2 - loss: 4139.222398519516
Time: 337.6191665229999
Epoch 3 - loss: 4138.759005069733
Time: 336.7097596900003
Epoch 4 - loss: 4138.750609606504
Time: 337.88217956100016
Epoch 5 - loss: 4138.613515347242
Time: 337.1083042249993
Epoch 6 - loss: 4138.712310671806
Time: 339.8560352859986
6
Val:
tensor(1.0599)
Test:
tensor(1.0599)


In [ ]:
args['epochs'] = 8
model = train(trainloader, layers)
print(8)
print('Val:')
print(evaluate(model, valoader))
print('Test:')
print(evaluate(model, testloader))

Epoch 1 - loss: 4138.038381099701
Time: 339.12343950600007
Epoch 2 - loss: 4138.0985932946205
Time: 337.14608776500063
Epoch 3 - loss: 4138.20110604167
Time: 337.49118975499914
Epoch 4 - loss: 4138.158048868179
Time: 337.1486140249999
Epoch 5 - loss: 4137.753827601671
Time: 338.16831340299905
Epoch 6 - loss: 4138.287654042244
Time: 336.5233725829985
Epoch 7 - loss: 4138.065690338612
Time: 337.7555926969999
Epoch 8 - loss: 4137.584454208612
Time: 337.42159865600115
8
Val:
tensor(1.0610)
Test:
tensor(1.0610)


In [ ]:
4137.584454208612/len(trainloader)

1.0587473014863389